## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
from tqdm.notebook import trange
import torch
from vjf import online
from scipy.stats import special_ortho_group

## Set seed(s)

In [ ]:
seed = 0

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

rng = np.random.default_rng(seed)

## Generate data

In [ ]:
%matplotlib inline

total_time = 1000
seconds_per_rotation = 1
samples_per_second = 10

theta = np.linspace(0, total_time/seconds_per_rotation*2*np.pi, total_time*samples_per_second)
x = np.column_stack([np.cos(theta), np.sin(theta)])
y = x @ special_ortho_group(dim=10, seed=rng).rvs(1)[:2]
y = y + rng.normal(0, 0.1, size=y.shape)
u = np.zeros((y.shape[0], 1))


In [ ]:
# hyperparameters
xdim = x.shape[-1]
ydim = y.shape[-1]
udim = u.shape[-1]

device = 'cpu' # 'cuda' does not work
likelihood='gaussian' # Gaussian observation
dynamics = 'rbf'  # RBF network dynamic model
recognizer = "mlp"  # MLP recognitiom model
rdim = 50  # number of RBFs
hdim = 100  # number of MLP hidden units

config=dict(
    resume=False,
    xdim=xdim,
    ydim=ydim,
    udim=udim,
    Ydim=udim,
    Udim=udim,
    rdim=rdim,
    hdim=hdim,
    lr=1e-3,
    clip_gradients=5.0,
    debug=False,
    likelihood=likelihood,  # 
    system=dynamics,
    recognizer=recognizer,
    random_seed=seed,
    C=(None, True),  # loading matrix: (initial, estimate)
    b=(None, True),  # bias: (initial, estimate)
    A=(None, False),  # transition matrix if LDS
    B=(np.zeros((xdim, udim)), False),  # interaction matrix
    Q=(1.0, True),  # state noise
    R=(1.0, True),  # observation noise
)



## minimal run

In [ ]:

mdl = online.VJF(config)

ys = torch.from_numpy(y).float()
us = torch.from_numpy(u).float()

mu = torch.zeros(ys.shape[0], xdim)
q = None  # current state

for i in np.arange(ys.shape[0]):
    q, _ = mdl.feed((ys[i:i+1], us[i:i+1]), q0=q)
    mu[i,:], _ = q

In [ ]:
fig, ax = plt.subplots()
ax.plot(*mu[-200:].detach().numpy().T)
ax.axis('equal')

## run with log_pred_p evaluation

In [ ]:
def diagonal_normal_logpdf(mean, variance, sample):
    mean = mean.flatten()
    variance = variance.flatten()
    sample = sample.flatten()
    
    assert len(mean) == len(variance) == len(sample), f"inconsistent shape: {mean.shape}, {variance.shape}, {sample.shape}"
    
    logprobs = []
    for i in range(len(sample)):
        x = sample[i]
        m = mean[i]
        v = variance[i]
        logprobs.append(-0.5 * ((x - m) ** 2 / v + np.log(2 * np.pi * v)))
    return sum(logprobs)


In [ ]:
def generate_samples(q, S, rng):
    filtering_mu, filtering_logvar = q
    
    mu_f = filtering_mu[0].detach().cpu().numpy().T
    var_f = filtering_logvar[0].detach().exp().cpu().numpy().T
    Sigma_f = np.eye(xdim) * var_f

    x = multivariate_normal(mu_f.flatten(), Sigma_f).rvs(size=S, random_state=rng).astype(np.float32)
    x = torch.from_numpy(x).to(device)
    return x

def step(x, mdl, rng):
    x += mdl.system.velocity(x) + mdl.system.noise.var ** 0.5 * torch.from_numpy(rng.normal(size=x.shape))
    return x

def get_logprob_and_distance(mdl, x, y_true):
    y_var = mdl.likelihood.logvar.detach().exp().cpu().numpy().T
    
    y_tilde = mdl.decoder(x).detach().cpu().numpy()
    
    sample_logprobs = [diagonal_normal_logpdf(y_est, y_var, y_true) for y_est in y_tilde]
    logprob = logsumexp(sample_logprobs) - np.log(x.shape[0])
    
    distance = np.linalg.norm(y_tilde - y_true, axis=-1).mean()
    
    return logprob, distance

def log_step(q, mdl, ys, t, rng, S=1000, T=10):
    x = generate_samples(q, S, rng)
    
    logprobs = []
    distances = []
    for i in range(T):
        if t + i < ys.shape[0]:
            y_tprime = ys[t + i].cpu().numpy()
        else:
            y_tprime = ys[t].cpu().numpy() * np.nan

        x = step(x, mdl, rng)
        logprob, distance = get_logprob_and_distance(mdl, x, y_tprime)

        logprobs.append(logprob)
        distances.append(distance)


    return logprobs, distances


In [ ]:
mdl = online.VJF(config).to(device)

ys = torch.from_numpy(y).float().to(device)
us = torch.from_numpy(u).float().to(device)

# could be None except for the logging functions
q = torch.zeros(1, xdim, device=device), torch.zeros(1, xdim, device=device)


logprobs = []
distances = []
mu2 = np.zeros((ys.shape[0], xdim))

for t in trange(ys.shape[0]):
    step_logprobs, step_distances = log_step(q, mdl, ys, t, rng, T=10)
    logprobs.append(step_logprobs)
    distances.append(step_distances)

    y_t = ys[t].unsqueeze(0)
    u_t = us[t].unsqueeze(0)
    q, loss = mdl.feed((y_t, u_t), q)
    
    mu2[t] = q[0].detach().numpy()

logprobs, distances = np.array(logprobs), np.array(distances)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(nrows=2)

for i in range(logprobs.shape[-1]):
    axs[0].plot(logprobs[:, i], label=f"{i+1} step{'s' if i > 0 else ''} ahead")
    axs[1].plot(distances[:, i], label=f"{i+1} step{'s' if i > 0 else ''} ahead")
    
for ax in axs:
    ax.set_xlabel("time")

axs[0].set_ylabel("log probability")
axs[1].set_ylabel("average prediction distance")
axs[1].legend(bbox_to_anchor=(1.01, 0.95))
# plt.ylim([-300, 0])

In [ ]:
# Checks that the randomness is actually controlled by SEED.
# This method is obviously hacky, but I'm keeping it because the logprobs
# and distances were inconsistent between runs before, despite seeding.

vars = {}
for var in ['seed', 'x', 'y', 'mu', 'ys', 'mu2', 'distances', 'logprobs']:
    v = globals()[var]
    if isinstance(v, torch.Tensor):
        v = v.detach().cpu().numpy()
    vars[var] = v

    s = f'/tmp/asdf_{var}'
    try:
        old_v = np.load(f"{s}.npy")
    except FileNotFoundError:
        old_v = None
    np.save(s, v)

    if old_v is not None:
        same = np.shape(v) == np.shape(old_v) and np.nanmax((v-old_v)**2) == 0
        print(f'{var}: {same}')
    else:
        print(f'{var}: NEW')
